In [3]:
# !pip install -r requirements.txt # in case you don't have some libraries required to run the code

In [4]:
import warnings
warnings.filterwarnings('ignore')

from utils import *

import pandas as pd
from tqdm.auto import tqdm
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from scipy.special import logit

tqdm.pandas()

Загрузим данные из первого задания (на этот раз чуть больше колонок) и переиспользуем модель из первого задания

In [5]:
types = {
    'os_id': str,
    'country_id': str,
    'zone_id': str,
    # 'campaign_clicks': int, – not required in this h/w
    'clicks': int,
    'banner_id': str,
    'banner_id0': str,
    'rate0': float,
    'g0': float,
    'coeff_sum0': float,
    'banner_id1': str,
    'rate1': float,
    'g1': float,
    'coeff_sum1': float
}
columns = ['date_time', *types.keys()]
df = pd.read_csv(
    '../data/data.csv',
    dtype=types,
    usecols=columns,
    parse_dates=[columns[0]]
)
df = df.set_index(columns[0]).sort_index()
df.head()

,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
date_time,,,,,,,,,,,,,
2021-09-01 00:02:49,30,596,0,7,596,0.050,0.064560,-4.312062,603,0.050,0.056510,-4.370191,0
2021-09-26 00:00:00,41,29,3,0,29,0.002,0.016386,-4.736584,6,0.002,0.020875,-4.898257,0
2021-09-26 00:00:00,1,188,2,15,188,0.008,0.014186,-3.811444,11754925,0.330,0.012351,-2.106896,0
2021-09-26 00:00:00,17,52,2,5,52,0.008,0.013550,-4.317590,41,0.004,0.067812,-3.739501,0
2021-09-26 00:00:00,47,73,4,13,73,0.008,0.120974,-2.382508,1040,0.008,0.157515,-3.037939,0


In [6]:
train, test0 = (
    df.loc[:'2021-10-01'].copy(),
    df.loc['2021-10-02'].copy()
)
ohe, target = (
    OneHotEncoder().fit(train[['os_id','country_id']]),
    TargetEncoder().fit(train[['zone_id','banner_id']], train.clicks)
)
train = feature_engineering(train, ohe, target)
train.head()

,hour,zone_id,banner_id,clicks,weekday,os_id_1,os_id_2,os_id_3,os_id_4,os_id_5,...,country_id_8,country_id_9,country_id_10,country_id_11,country_id_12,country_id_13,country_id_14,country_id_15,country_id_16,country_id_17
date_time,,,,,,,,,,,,,,,,,,,,,
2021-09-01 00:02:49,0,0.021063,0.017529,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-26 00:00:00,0,0.007601,0.024826,0,6,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-26 00:00:00,0,0.038422,0.013548,0,6,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-26 00:00:00,0,0.027595,0.020776,0,6,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-09-26 00:00:00,0,0.018115,0.091559,0,6,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
model = create_model(train, c=1.0)

Посчитаем pi_0, pi_1

In [8]:
test0 = test0[test0.banner_id == test0.banner_id0]
test1 = test0.copy()
d = np.sqrt(test0.g0 ** 2 + test0.g1 ** 2)
e1 = test0.coeff_sum0 - test0.coeff_sum1
test1.banner_id = test1.banner_id1
test0, test1 = (
    feature_engineering(test0, ohe, target),
    feature_engineering(test1, ohe, target)
)

In [9]:
pi_0 = get_pi(e=e1, d=d)
print(pi_0)

[0.12639264 0.00141389 1.         ... 0.         0.43539021 1.        ]


In [10]:
test0_predictions = logit(model.predict_proba(test0.drop(columns="clicks").values)[:, 1])
test1_predictions = logit(model.predict_proba(test1.drop(columns="clicks").values)[:, 1])
e2 = test0_predictions - test1_predictions

In [11]:
pi_1 = get_pi(e=e2, d=d)
print(pi_1)

[0.5        0.20578663 1.         ... 0.         0.5        1.        ]


Наконец посчитаем CIPS

In [12]:
get_cips(pi_0, pi_1, test0)

0.07471744806183095

Для оптимизации регуляризации с CIPS достаточно просто запустить перебор по определенному рейнджу параметра с оглядкой на CIPS :) Но так вышло, что времени это посчитать в этом ноутбуке не нашлось